In [2]:
%matplotlib notebook
import numpy as np
from proj1_helpers import predict_labels,load_csv_data, create_csv_submission
from helpers import *
import matplotlib.pyplot as plt

cheminTrain = 'C:/Users/Raphael/Desktop/EPFL/Machine_Learning/train.csv'
cheminTest =  'C:/Users/Raphael/Desktop/EPFL/Machine_Learning/test.csv'

yb, input_data, ids = load_csv_data(cheminTrain)
yb_test, input_data_test, ids_test = load_csv_data(cheminTest)

x,_,_ = standardize(input_data)
x_test,_,_ = standardize(input_data_test)

In [3]:
def compute_mse(y, tx, w):
    return (1/(2*tx.shape[0]))*np.linalg.norm(y-tx@w)**2

In [4]:
def least_squares(y, tx):
    txt = tx.transpose()
    w= np.linalg.solve(txt@tx,txt@y)
    loss = compute_mse(y,tx,w)
    
    return loss, w

In [8]:
loss, w = least_squares(yb,x)
print(loss)
print('\n')
print(w)
print('\n')
prediction(w,x,yb)

0.388952314937


[  2.93788271e-02  -2.52531475e-01  -2.54791124e-01  -3.03696823e-02
  -1.40144681e+00   2.95701640e-01  -1.07889471e+01   2.67880862e-01
  -2.44934975e-03  -3.28822480e+02  -1.82647888e-01   1.14039627e-01
   2.05045954e+01   6.38843559e+01  -3.18961917e-04  -1.80884297e-03
   6.29935231e+01  -4.48641512e-04   1.54379291e-03   1.21462701e-01
   3.95268787e-04  -6.33223473e-02  -2.06747093e-01  -1.16655767e-01
   9.86256640e-02   1.67907688e-01  -3.35146213e-02  -2.98358677e+00
  -5.36388090e+00   2.78474640e+02]


Good prediction :  180029
Bad predition :  69971
Ratio :  0.720116


In [7]:
def split_data(x, y, ratio, seed=1):

    np.random.seed(seed)
    N = len(x)

    n_train = int(ratio * N)
    
    train_index = np.random.choice(N, n_train, replace=False)

    index = np.arange(N)

    mask = np.in1d(index, train_index)

    test_index = np.random.permutation(index[~mask])

    x_train = x[train_index]
    y_train = y[train_index]

    x_test = x[test_index]
    y_test = y[test_index]

    return x_train, y_train, x_test, y_test

def prediction(w_train,tx,y_test):
    
    y_pred1 = predict_labels(w_train,tx)

    for n,i in enumerate(y_test):
        if i==0:
              y_test[n]=-1
            
    right = 0
    wrong = 0

    for i in range(len(y_test)):
        if y_test[i] == y_pred1[i]:
            right +=1 
        else:
            wrong +=1 
    
        
    print("Good prediction : ", right)
    print("Bad predition : " , wrong)
    print("Ratio : " ,right/len(y_test))

# Compare overfit on least squares and ridge

In [9]:
def ridge_regression(y, tx, lambda_):
    txt = tx.transpose()
    lambda_prim = 2*y.shape[0]*lambda_
    identity = np.identity(tx.shape[1])
    w = np.linalg.inv(txt@tx+lambda_prim*identity)@txt@y
    loss = compute_mse(y,tx,w)
    return loss,w
def build_poly(x, degree):
    y = np.copy(x)
    for i in range(1,degree):
        y = np.concatenate((y,np.power(x,i+1)),axis=1)
                           
    return y

In [19]:
deg = 3
lambda_ = 0.008531678524172814
x_train, y_train, x_test, y_test = split_data(x, yb, 0.8)
x_to_deg = build_poly(x_train,deg)
x_to_deg_test = build_poly(x_test,deg)


print('LEAST SQR')
loss_ls, w_ls = least_squares(y_train,x_to_deg)
print(loss_ls)
print('\n')
print(w_ls)
print('\n')
prediction(w_ls,x_to_deg_test,y_test)

print('\n\n')
print('RIDGE R.')
loss_rr, w_rr = ridge_regression(y_train,x_to_deg,lambda_)
print(loss_rr)
print('\n')
print(w_rr)
print('\n')
prediction(w_rr,x_to_deg_test,y_test)

LEAST SQR
0.310624427564


[  7.90572620e-01  -2.29644905e-01  -3.47061768e-01   1.18116407e-01
   1.83070084e+04  -9.05475762e-01   2.61217945e+05   2.37953262e-01
  -3.04868618e-02  -4.48685881e+02  -2.37616255e-01  -8.43610091e-02
  -9.54903945e+04   8.71399244e+01  -3.35421373e-04   6.52878210e-03
   8.58456401e+01   2.37314147e-03   1.08129274e-02  -8.05879975e-03
   5.08798919e-03  -2.41069120e-02  -8.31731341e+04   1.47675491e+01
   1.25751155e+05  -5.59550616e+04   8.28302436e+00   8.42289125e+04
  -2.82470061e+05   3.80060044e+02  -6.69459615e-02   6.14367548e-02
   1.55704702e-02   2.74160392e-02  -1.87441610e+04   2.76667690e-01
  -1.66297378e+05  -6.69766560e-02   6.53395451e-03  -1.08215474e-02
   4.90159213e-02   1.69188322e-02   1.95527828e+04  -3.30197025e-02
  -3.43031017e-02  -9.95021233e-04  -4.39377234e-02  -5.51388757e-02
  -2.46795709e-03   3.92958701e-03  -2.29946583e-03  -3.89958596e-02
   1.76485363e+05  -1.24810326e+01  -1.64094087e+05   6.88602394e+04
  -2.76

In [21]:
lambdas = np.logspace(-15, 0, 30)
for ind, lambda_ in enumerate(lambdas):
    loss, w = ridge_regression(y_train,x_to_deg,lambda_)
    prediction(w,x_to_deg_test,y_test)
    print("Current lambda = {i}".format(i=lambda_))

Good prediction :  30001
Bad predition :  19999
Ratio :  0.60002
Current lambda = 1e-15
Good prediction :  34400
Bad predition :  15600
Ratio :  0.688
Current lambda = 3.290344562312671e-15
Good prediction :  35315
Bad predition :  14685
Ratio :  0.7063
Current lambda = 1.0826367338740562e-14
Good prediction :  35152
Bad predition :  14848
Ratio :  0.70304
Current lambda = 3.562247890262451e-14
Good prediction :  35137
Bad predition :  14863
Ratio :  0.70274
Current lambda = 1.1721022975334794e-13
Good prediction :  35160
Bad predition :  14840
Ratio :  0.7032
Current lambda = 3.8566204211634723e-13
Good prediction :  35124
Bad predition :  14876
Ratio :  0.70248
Current lambda = 1.2689610031679235e-12
Good prediction :  35091
Bad predition :  14909
Ratio :  0.70182
Current lambda = 4.175318936560409e-12
Good prediction :  34995
Bad predition :  15005
Ratio :  0.6999
Current lambda = 1.373823795883261e-11
Good prediction :  34895
Bad predition :  15105
Ratio :  0.6979
Current lambda = 

In [18]:
print(np.mean(w_ls))
print(np.mean(w_rr))

39774.0235781
-29622.730225
